# Création du dataset final

## Import des bibiliothèques

In [1]:
import pandas as pd
import os
import glob
from datetime import datetime
from pathlib import Path

## Récupération des données brutes dans le répertoire /data/raw

In [2]:
repertoire_data_raw = Path.cwd().parent.parent / "data" / "raw"

fichiers_csv = [f for f in list(repertoire_data_raw.glob("*.csv")) if "metadata" not in f.name.lower() and "~" not in f.name]

for file_path in fichiers_csv:
    print(f"Lecture du fichier CSV : {file_path.name}")

fichiers_excel = [f for f in list(repertoire_data_raw.glob("*.xlsx")) if "metadata" not in f.name.lower() and "~" not in f.name]

for file_path in fichiers_excel:
    print(f"Lecture du fichier Excel : {file_path.name}")

Lecture du fichier CSV : défaillances_entreprises-tous_secteurs-brut-france-1990_2025.csv
Lecture du fichier CSV : euribor-3_month-1994_2025.csv
Lecture du fichier CSV : indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025.csv
Lecture du fichier CSV : indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025.csv
Lecture du fichier CSV : indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025.csv
Lecture du fichier CSV : indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025.csv
Lecture du fichier CSV : ipc-glissement_annuel-ensemble_menage-1991_2025.csv
Lecture du fichier CSV : ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025.csv
Lecture du fichier CSV : isj-glissement_annuel-ensemble_menage-1991_2025.csv
Lecture du fichier CSV : main_refinancing_operations-1999_2025.csv
Lecture du fichier CSV : ocde-taux_chomage_mensuel-1984_2025.csv
Lecture du fichier CSV : pib_total-volume_prix_année_préc

In [3]:
# Dictionnaire pour stocker les dataframes
dict_dfs = {}

# Importation des CSV
for file_path in fichiers_csv:
    # On utilise le nom du fichier (sans .csv) comme clé
    nom_cle = file_path.stem 
    print(f"Importation de : {nom_cle}")
    # Lecture personnalisée (ajustez sep et encoding selon vos tests précédents)
    if nom_cle == 'euribor-3_month-1994_2025' or nom_cle == 'main_refinancing_operations-1999_2025' or nom_cle == 'ocde-taux_chomage_mensuel-1984_2025':
        df = pd.read_csv(file_path, sep=',', encoding='latin1')
    else:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    # Stockage dans le dictionnaire
    dict_dfs[nom_cle] = df

# Importation des EXCEL
for file_path in fichiers_excel:
    nom_cle = file_path.stem 
    print(f"Importation de : {nom_cle}")
    # Lecture personnalisée (ajustez sep et encoding selon vos tests précédents)
    if nom_cle == 'séries_CVS_T22025':
        df = pd.read_excel(file_path, sheet_name='DEFM cat ABCDE',header=6)
    elif nom_cle == 'interimaire-evolution-trimestrielle-2000_2025':
        df = pd.read_excel(file_path, sheet_name="Nombre d'intérimaires",header=7)        
    else:
        df = pd.read_excel(file_path, sheet_name=0)

    # Stockage dans le dictionnaire
    dict_dfs[nom_cle] = df


    
# Pour voir la liste des fichiers chargés :
#print("\nFichiers chargés :", dict_dfs.keys())

Importation de : défaillances_entreprises-tous_secteurs-brut-france-1990_2025
Importation de : euribor-3_month-1994_2025
Importation de : indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025
Importation de : indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025
Importation de : indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025
Importation de : indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025
Importation de : ipc-glissement_annuel-ensemble_menage-1991_2025
Importation de : ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025
Importation de : isj-glissement_annuel-ensemble_menage-1991_2025
Importation de : main_refinancing_operations-1999_2025
Importation de : ocde-taux_chomage_mensuel-1984_2025
Importation de : pib_total-volume_prix_année_précédente_chaînés-série_CVS-CJO-1949_2025
Importation de : interimaire-evolution-trimestrielle-2000_2025
Importation de : séries_CVS_T22025
Imp

## Création du dataframe template dans lequel on ajoutera les données des fichiers brutes

In [4]:
# 1. Définir le début et la fin (aujourd'hui)
date_debut = "1990-01-01"
date_fin = datetime.now()

# 2. Créer la plage de dates mensuelle
# 'MS' signifie 'Month Start' (1er du mois)
# 'ME' signifierait 'Month End' (dernier jour du mois)
date_periode = pd.date_range(start=date_debut, end=date_fin, freq='MS')

# 3. Créer le DataFrame
df = pd.DataFrame({'date': date_periode})

# 4. Extraire les colonnes demandées précédemment (Année, Mois, Trimestre)
df['annee'] = df['date'].dt.year
df['trimestre'] = df['date'].dt.quarter
df['mois'] = df['date'].dt.month

# Traitement des données brutes

## 1) Demandeurs d'emploi inscrits à France Travail, en catégories A, B, C, D, E (DEFM ABCDE)

In [5]:
# Demandeurs d'emploi inscrits à France Travail en catégories A, B, C, D, E - DEFM ABCDE
df_defm_abcde = dict_dfs['séries_CVS_T22025']

# Renommer les colonnes pour les uniformiser
df_defm_abcde = df_defm_abcde.rename(columns={
    'DEFM ABCDE': 'nb_demandeurs_emploi_abcde'
})

df_defm_abcde['time_period'] = pd.to_datetime(df_defm_abcde['Période'].str.replace('T', '-Q'))
#df_defm_abcde['time_period'] = pd.to_datetime(df_defm_abcde['time_period'], dayfirst=True)
df_defm_abcde['date_jointure'] = df_defm_abcde['time_period'].dt.to_period('M').dt.to_timestamp()


# Fusion des deux dataframes
df_final = pd.merge(
    df, 
        df_defm_abcde[['date_jointure', 'nb_demandeurs_emploi_abcde']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_52312\4160552738.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_defm_abcde['time_period'] = pd.to_datetime(df_defm_abcde['Période'].str.replace('T', '-Q'))


In [6]:
#df_final[df_final['annee'] ==2025]

## 2) Offres d'emploi diffusées à France Travail (OEDFT)

In [7]:
df_oedft = dict_dfs['séries_offres_difusees_T32025']

#Renommer les colonnes pour les uniformiser
df_oedft = df_oedft.rename(columns={
    'Nombre d\'offres diffusées': 'nb_offres_france_travail'
})

# Création d'une colonne date aggrégeant les différentes infos de date disponibles
df_oedft['day'] = 1
df_oedft['time_period'] = pd.to_datetime(pd.DataFrame({
    'year': df_oedft['Année'],
    'month': df_oedft['Mois'],
    'day': df_oedft['day']
}))

df_oedft['time_period'] = pd.to_datetime(df_oedft['time_period'], dayfirst=True)
df_oedft['date_jointure'] = df_oedft['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_oedft[['date_jointure', 'nb_offres_france_travail']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 3) Population active et chômage (PAC)

In [8]:
## TODO

## 4) Taux de chômage par pays (OCDE)

In [9]:
# ocde-taux_chomage_mensuel-1984_2025
df_ocde = dict_dfs['ocde-taux_chomage_mensuel-1984_2025']

# Filtrage des données sur la FRANCE
df_ocde = df_ocde[df_ocde['REF_AREA'] == 'FRA']

# Renommer les colonnes pour les uniformiser
df_ocde = df_ocde.rename(columns={
    'OBS_VALUE': 'taux_chomage_ocde'
})

df_ocde['time_period'] = pd.to_datetime(df_ocde['TIME_PERIOD'], dayfirst=True)
df_ocde['date_jointure'] = df_ocde['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ocde[['date_jointure', 'taux_chomage_ocde']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_52312\2262536533.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ocde['time_period'] = pd.to_datetime(df_ocde['TIME_PERIOD'], dayfirst=True)


## 5) Euribor 3-month (E3M)

In [10]:
# euribor-3_month-1994_2025
df_e3m = dict_dfs['euribor-3_month-1994_2025']

# Renommer les colonnes pour les uniformiser
df_e3m = df_e3m.rename(columns={
    'Euribor 3-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA)': 'taux_euribor_3m'
})

df_e3m['time_period'] = pd.to_datetime(df_e3m['DATE'], dayfirst=True)
df_e3m['date_jointure'] = df_e3m['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_e3m[['date_jointure', 'taux_euribor_3m']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_52312\632589263.py:9: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_e3m['time_period'] = pd.to_datetime(df_e3m['DATE'], dayfirst=True)


## 6) Main refinancing operations (MRO)

In [11]:
# main_refinancing_operations -1999_2025
df_mro = dict_dfs['main_refinancing_operations-1999_2025']

# Renommer les colonnes pour les uniformiser
df_mro = df_mro.rename(columns={
    'Main refinancing operations - fixed rate tenders (fixed rate) (date of changes) - Level (FM.B.U2.EUR.4F.KR.MRR_FR.LEV)': 'MRO'
})

df_mro['time_period'] = pd.to_datetime(df_mro['DATE'], format='%Y-%m-%d')
df_mro['date_jointure'] = df_mro['time_period'].dt.to_period('M').dt.to_timestamp()


# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_mro[['date_jointure', 'MRO']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 7) Indice des prix à la consommation (IPC)

In [12]:
# ipc-glissement_annuel-ensemble_menage-1991_2025
df_ipc = dict_dfs['ipc-glissement_annuel-ensemble_menage-1991_2025']

# Renommer les colonnes pour les uniformiser
df_ipc = df_ipc.rename(columns={
    'obs_value': 'ipc'
})

df_ipc['time_period'] = pd.to_datetime(df_ipc['time_period'], dayfirst=True)
df_ipc['date_jointure'] = df_ipc['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ipc[['date_jointure', 'ipc']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 8) Inflation sous-jacente (ISJ)

In [13]:
# isj-glissement_annuel-ensemble_menage-1991_2025
df_isj = dict_dfs['isj-glissement_annuel-ensemble_menage-1991_2025']

# Renommer les colonnes pour les uniformiser
df_isj = df_isj.rename(columns={
    'obs_value': 'isj'
})

df_isj['time_period'] = pd.to_datetime(df_isj['time_period'], dayfirst=True)
df_isj['date_jointure'] = df_isj['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_isj[['date_jointure', 'isj']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 9) Indice des prix à la consommation ENERGIE (IPC ENERGIE)

In [14]:
# ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025
df_ipc_energie = dict_dfs['ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025']

# Renommer les colonnes pour les uniformiser
df_ipc_energie = df_ipc_energie.rename(columns={
    'obs_value': 'ipc_energie_only'
})

df_ipc_energie['time_period'] = pd.to_datetime(df_ipc_energie['time_period'], dayfirst=True)
df_ipc_energie['date_jointure'] = df_ipc_energie['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ipc_energie[['date_jointure', 'ipc_energie_only']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 10) Indicateur du climat des affaires (ICA)

In [15]:
# indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025
df_ica = dict_dfs['indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025']

# Renommer les colonnes pour les uniformiser
df_ica = df_ica.rename(columns={
    'obs_value': 'indicateur_climat_affaires'
})

df_ica['time_period'] = pd.to_datetime(df_ica['time_period'], dayfirst=True)
df_ica['date_jointure'] = df_ica['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ica[['date_jointure', 'indicateur_climat_affaires']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 11) Indicateur du climat de l'emploi (ICE)

In [16]:
# indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025
df_ice = dict_dfs['indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025']

# Renommer les colonnes pour les uniformiser
df_ice = df_ice.rename(columns={
    'obs_value': 'indicateur_climat_emploi'
})

df_ice['time_period'] = pd.to_datetime(df_ice['time_period'], dayfirst=True)
df_ice['date_jointure'] = df_ice['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ice[['date_jointure', 'indicateur_climat_emploi']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 12) Indicateur de retournement conjoncturel (IRC)

In [17]:
# indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025
df_irc = dict_dfs['indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025']

# Renommer les colonnes pour les uniformiser
df_irc = df_irc.rename(columns={
    'obs_value': 'indicateur_retournement_conjoncturel'
})

df_irc['time_period'] = pd.to_datetime(df_irc['time_period'], dayfirst=True)
df_irc['date_jointure'] = df_irc['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_irc[['date_jointure', 'indicateur_retournement_conjoncturel']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 13) Produit intérieur brut total (PIB)

In [18]:
# pib_total-volume_prix_année_précédente_chaînés-série_CVS-CJO-1949_2025
df_pib = dict_dfs['pib_total-volume_prix_année_précédente_chaînés-série_CVS-CJO-1949_2025']

# Renommer les colonnes pour les uniformiser
df_pib = df_pib.rename(columns={
    'obs_value': 'pib'
})

df_pib['time_period'] = pd.to_datetime(df_pib['time_period'].str.replace('-T', 'Q'))

df_pib['date_jointure'] = df_pib['time_period'].dt.to_period('M').dt.to_timestamp()

df_final = pd.merge(
    df_final, 
    df_pib[['date_jointure', 'pib']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_52312\1229989185.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pib['time_period'] = pd.to_datetime(df_pib['time_period'].str.replace('-T', 'Q'))


## 14) Indice du coût du travail (ICT)

In [19]:
# indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025
df_ict = dict_dfs['indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025']

# Renommer les colonnes pour les uniformiser
df_ict = df_ict.rename(columns={
    'obs_value': 'ict'
})

df_ict['time_period'] = pd.to_datetime(df_ict['time_period'].str.replace('-T', 'Q'))
df_ict['date_jointure'] = df_ict['time_period'].dt.to_period('M').dt.to_timestamp()

df_final = pd.merge(
    df_final, 
    df_ict[['date_jointure', 'ict']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_52312\558528430.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ict['time_period'] = pd.to_datetime(df_ict['time_period'].str.replace('-T', 'Q'))


## 15) Défaillances d'entreprises (DE)

In [20]:
# défaillances_entreprises-tous_secteurs-brut-france-1990_2025
df_de = dict_dfs['défaillances_entreprises-tous_secteurs-brut-france-1990_2025']

# Renommer les colonnes pour les uniformiser
df_de = df_de.rename(columns={
    'obs_value': 'nb_defaillances_entreprise'
})

df_de['time_period'] = pd.to_datetime(df_de['time_period'], dayfirst=True)
df_de['date_jointure'] = df_de['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_de[['date_jointure', 'nb_defaillances_entreprise']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 16) L'emploi intérimaire (EI)

In [21]:
# indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025
df_ei = dict_dfs['interimaire-evolution-trimestrielle-2000_2025']

# Renommer les colonnes pour les uniformiser
df_ei = df_ei.rename(columns={
    'Niveau': 'nb_interimaires'
})

serie_trimestres = df_ei.iloc[:, 1]
df_ei['time_period'] = pd.PeriodIndex(df_ei.iloc[:, 0].astype(str).str.replace('T', 'Q'), freq='Q').to_timestamp()
df_ei['date_jointure'] = df_ei['time_period'].dt.to_period('M').dt.to_timestamp()

df_final = pd.merge(
    df_final, 
    df_ei[['date_jointure', 'nb_interimaires']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## Visualisation du dataframe final

In [22]:
df_final.tail(20)

,date,annee,trimestre,mois,demandeurs_emploi_abcde,nb_offres_france_travail,taux_chomage_ocde,taux_euribor_3m,MRO,ipc,isj,ipc_energie_only,indicateur_climat_affaires,indicateur_climat_emploi,indicateur_retournement_conjoncturel,pib,ict,nb_defaillances_entreprise,nb_interimaires
415,2024-06-01,2024,2,6,NaN,923400.0,7.4,3.7245,4.25,2.2,1.8,4.8,99.6,100.1,-0.03,NaN,NaN,5708.0,NaN
416,2024-07-01,2024,3,7,6174500.0,1050800.0,7.4,3.6848,NaN,2.3,1.5,8.5,94.4,97.2,-1.00,655027.0,110.1,5843.0,726325.294868
417,2024-08-01,2024,3,8,NaN,760200.0,7.4,3.5476,NaN,1.8,1.7,0.4,96.6,98.4,0.70,NaN,NaN,1599.0,NaN
418,2024-09-01,2024,3,9,NaN,876100.0,7.4,3.4337,3.65,1.1,1.4,-3.3,97.4,98.6,-0.80,NaN,NaN,6314.0,NaN
419,2024-10-01,2024,4,10,6251200.0,1023400.0,7.3,3.1666,3.40,1.2,1.4,-2.0,97.0,97.4,-0.90,654734.0,111.3,6554.0,713974.330136
420,2024-11-01,2024,4,11,NaN,863400.0,7.3,3.0068,NaN,1.3,1.5,-0.7,96.1,98.4,-0.70,NaN,NaN,6099.0,NaN
421,2024-12-01,2024,4,12,NaN,860800.0,7.3,2.8161,3.15,1.3,1.3,1.2,95.0,96.5,-0.90,NaN,NaN,5752.0,NaN
422,2025-01-01,2025,1,1,6494600.0,836700.0,7.3,2.7031,NaN,1.7,1.4,2.7,95.4,97.0,-0.70,655389.0,112.3,5948.0,710419.927973
423,2025-02-01,2025,1,2,NaN,772600.0,7.5,2.5250,2.90,0.8,1.3,-5.8,95.9,93.5,-0.60,NaN,NaN,5826.0,NaN
424,2025-03-01,2025,1,3,NaN,873400.0,7.6,2.4424,2.65,0.8,1.3,-6.6,96.6,96.4,-0.60,NaN,NaN,6382.0,NaN


## Export du dataset

In [23]:
repertoire_data_processed = Path.cwd().parent.parent / "data" / "processed"
chemin_export = repertoire_data_processed / "dataset.csv"

df_final.to_csv(chemin_export, index=False, encoding='utf-8')